In [ ]:
import os
import cv2
import numpy as np

In [ ]:
# Specify the paths for the 2 files

cwd = os.getcwd()
protoFile = "{}/pose_deploy_linevec_faster_4_stages.prototxt".format(cwd)
weightsFile = "{}/pose_iter_440000.caffemodel".format(cwd)

# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [ ]:
#구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read image
frame = cv2.imread("test1.png")
print(frame.shape)
# Specify the input image dimensions
inWidth = 299
inHeight = 299
frameWidth = frame.shape[1]
frameHeight = frame.shape[0]
threshold = 0.1

nPoints = 10
# Forward training set into OpenPose model to generate output
inWidth = 299
inHeight = 299
# Prepare the frame to be fed to the network
inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=True, crop=False)
 
# Set the prepared object as the input blob of the network
net.setInput(inpBlob)


In [ ]:
output = net.forward()
#output

In [ ]:
H = output.shape[2]
W = output.shape[3]
print(H,W)
# Empty list to store the detected keypoints
points = []
nPoints = 15
for i in [0,1,2,3,4,5,6,7,8]:#수정 사항
    # confidence map of corresponding body's part.
    probMap = output[0, i, :, :]
 
    # Find global maxima of the probMap.
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    # Scale the point to fit on the original image
    x = (frameWidth * point[0]) / W#수정사항
    y = (frameHeight * point[1]) / H
 
    if (prob > threshold):
        cv2.circle(frame, (int(x), int(y)), 15, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)
 
        # Add the point to the list if the probability is greater than the threshold
        points.append((int(x), int(y)))
    else :
        points.append(None)
from google.colab.patches import cv2_imshow



cv2_imshow(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7]]#, [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] 
for pair in POSE_PAIRS:
    partA = pair[0]
    partB = pair[1]
 
    if points[partA] and points[partB]:
        cv2.line(frame, points[partA], points[partB], (0, 255, 0), 3)
cv2_imshow(frame)

In [ ]:


cwd = os.getcwd()
# Specify the paths for the 2 files
protoFile = "{}/pose_deploy_linevec_faster_4_stages.prototxt".format(cwd)
weightsFile = "{}/pose_iter_160000.caffemodel".format(cwd)
nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
  
 # Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
frameWidth = 640
frameHeight = 480
threshold = 0.1
# Forward training set into OpenPose model to generate output
inWidth = 299
inHeight = 299
m = train_images.shape[0]
train_outputs = np.zeros((1,44,38,38))
for i in range(m):
    inpBlob = cv2.dnn.blobFromImage(train_images[i], 1.0/255,    (inWidth, inHeight),(0, 0, 0), swapRB=True, crop=False)
    net.setInput(inpBlob)
    output = net.forward()
    train_outputs = np.vstack((train_outputs,output))
   
   
outputs = np.delete(train_outputs,(0),axis=0)
H = train_outputs.shape[2]
W = train_outputs.shape[3]
print(train_outputs.shape)
# Generate keypoints for training set
m = 973
H = 38
W = 38
train_points = np.zeros((m,15,2))
for sample in range(m):
   for i in range(nPoints):
       # confidence map of corresponding body's part.
       probMap = train_outputs[sample, i, :, :]
# Find global maxima of the probMap.
       minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
# Scale the point to fit on the original image
       x = (frameWidth * point[0]) / W
       y = (frameHeight * point[1]) / H
       if prob > threshold :
           train_points[sample,i,0] = int(x)
           train_points[sample,i,1] = int(y)

In [ ]:
# Processed images with sticks on original image
train_processed = np.copy(train_images).astype(np.uint8)
for sample in range(m):
   for point in range(nPoints):
     if train_points[sample,point,0] !=0 and train_points[sample,point,1] !=0 :
       cv2.circle(train_processed[sample], (int(train_points[sample,point,0]), int(train_points[sample,point,1])), 10, (255,255,0), thickness=-1, lineType=cv2.FILLED)
     
     # draw lines 
     for pair in POSE_PAIRS:
       partA = pair[0]
       partB = pair[1]
if train_points[sample,partA,0] != 0 and train_points[sample,partA,1] != 0 and train_points[sample,partB,0] != 0 and train_points[sample,partB,1] != 0:
          cv2.line(train_processed[sample], (int(train_points[sample,partA,0]),int(train_points[sample,partA,1]))
                   , (int(train_points[sample,partB,0]),int(train_points[sample,partB,1])), (255,255,0), 3)
# Processed images with sticks on a black background
train_processed_grey = np.zeros((m,train_images.shape[1],train_images.shape[2],1)).astype(np.uint8)
for sample in range(m):
   for point in range(nPoints):
     if train_points[sample,point,0] !=0 and train_points[sample,point,1] !=0 :
       cv2.circle(train_processed_grey[sample], (int(train_points[sample,point,0]), int(train_points[sample,point,1])), 10, (1), thickness=50, lineType=cv2.FILLED)
     
     # draw lines 
     for pair in POSE_PAIRS:
       partA = pair[0]
       partB = pair[1]
if train_points[sample,partA,0] != 0 and train_points[sample,partA,1] != 0 and train_points[sample,partB,0] != 0 and train_points[sample,partB,1] != 0:
          cv2.line(train_processed_grey[sample], (int(train_points[sample,partA,0]),int(train_points[sample,partA,1]))
                   , (int(train_points[sample,partB,0]),int(train_points[sample,partB,1])), (1), 3)

In [ ]:
!pip install -Uqq fastbook
import fastai
from fastai.metrics import error_ratefrom torchvision.models import *
import pretrainedmodels
from fastai.callbacks.tracker import SaveModelCallback
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
bs = 8
# Importing of dataset
data = ImageDataBunch.from_folder(base_dir,train='train',valid='val', ds_tfms = get_transforms(), size =299, bs=bs).normalize(imagenet_stats)
#ResNet-50
models.resnet50
#ResNeXt-101
def resnext101_64x4d(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.resnext101_64x4d(pretrained=pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])
#PASNet-5
def identity(x): return x
def pnasnet5large(pretrained=True):    
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.pnasnet5large(pretrained=pretrained, num_classes=1000) 
    model.logits = identity
    return nn.Sequential(model)
# Training of model
learn = cnn_learner(data, model, metrics = accuracy)
learn.fit(20,callbacks=[SaveModelCallback(learn,monitor='accuracy',every="improvement",name ="top_acc")])
learn.lr_find()
learn.recorder.plot()
learn.unfreeze()
learn.fit_one_cycle(5,callbacks=[SaveModelCallback(learn,monitor='accuracy',every="improvement",name="top_acc_1")])